# Resolución de ejercicios: Capítulo 7
## Autores: Eduardo Cuadros, Juan Barrera


# Punto 3: Sistema RAG sencillo para acceder a información actualizada de la Universidad Nacional mediante un chat

Lo primero que se hace instalar las dependencias necesarias para implementar un modelo de lenguaje usando Ollama. Esto es necesario para establecer un entorno y ejecutar un modelo LLM localmente

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > ollama_serve.log 2>&1 &
!ollama pull llama3.1
!pip install -qU langchain_community

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


Se importa la clase `Ollama` desde la biblioteca

In [ ]:
from langchain_community.llms import Ollama

llm_llama = Ollama(model="llama3.1")

/tmp/ipython-input-4-2454314464.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm_llama = Ollama(model="llama3.1")


Se crea una lista que contiene varias URLs de páginas del área de Bienestar de la Universidad Nacional

In [ ]:
unal_urls = ["https://bienestar.unal.edu.co/sistema-de-bienestar/area-de-acompanamiento-integral/",
             "https://bienestar.unal.edu.co/sistema-de-bienestar/area-de-gestion-y-fomento-socioeconomico/",
             "https://bienestar.unal.edu.co/sistema-de-bienestar/area-de-salud/",
             "https://bienestar.unal.edu.co/sistema-de-bienestar/area-de-actividad-fisica-y-deporte/",
             "https://bienestar.unal.edu.co/sistema-de-bienestar/area-de-cultura/",
             "https://bienestar.unal.edu.co/sistema-de-bienestar/educacion-inclusiva/que-es-educacion-inclusiva/"]

Se descarga el modelo `nomic-embed-text`, el cual es especializado en la generación de embeddings.

In [ ]:
!ollama pull nomic-embed-text

Se crea una instancia que permite generar los embeddings vectoriales de texto, lo que será útil para indexar y recuperar los documentos.

In [ ]:
from langchain_ollama import OllamaEmbeddings

nomic_ollama_embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
!pip install -qU langchain_experimental langchain-ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install -qU chromadb langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.8 MB/s eta 0

Se crea una función para resetear una colección específica antes de cargar nueva información en ella.

In [ ]:
from langchain_chroma import Chroma

def reset_collection(collection_name, persist_directory):
  Chroma(collection_name=collection_name, embedding_function=nomic_ollama_embeddings, persist_directory=persist_directory).delete_collection()

In [ ]:
vector_store = Chroma(collection_name="gabo_rag", embedding_function=nomic_ollama_embeddings, persist_directory="chroma")
retriever = vector_store.as_retriever(search_kwargs={"k": 3})


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Se construye la cadena RAG, _Retrieval-Augmented Generation_. Es decir, el flujo que une la recuperación de documentos con la generación de respuestas.

También se le da el prompt principal, el cual se compone de la estructura que debe seguir y la postura que debe tomar.

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

system_prompt = (
    "Eres un representante de la Universidad Nacional de Colombia, la mejor universidad del país. Responde a las preguntas con el ánimo de dar información a los estudiantes y a los aspirantes\n"
    "En cada consulta recibirás una pregunta, la cual debes investigar en las diferentes áreas de la universidad.\n"
    "Responde preguntas estrictamente relacionadas con la universidad. \n"
    "Responde de manera precisa, consisa y relevante a la pregunta que se ha hecho, sin desviarte del tema y limitando tu respuesta a un parrafo. \n"
    "No debes inventar la información, si no tienes suficiente información para dar una respuesta precisa, simplemente di que no sabes la respuesta."
    "\nContexto:\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm_llama, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

Aqui empezamos a hacerle preguntas al RAG. Comenzamos preguntandole sobre su propósito

In [ ]:
question = "¿Quien eres?"

response = rag_chain.invoke({"input": question})
unal = response["answer"]

f"UNAL_RAG: '{unal}'"

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


"UNAL_RAG: 'Soy el representante de la Universidad Nacional de Colombia, la institución líder en educación superior del país. Mi función es brindar información y respuestas a las preguntas de los estudiantes y aspirantes que desean saber más sobre nuestros programas académicos, servicios ofrecidos y oportunidades de crecimiento profesional y personal dentro de nuestra comunidad universitaria. Estoy aquí para ayudarte con cualquier consulta relacionada con la Universidad Nacional de Colombia. ¿En qué puedo asistirte hoy?'"

Se le pregunta información sobre algunas páginas que se le entregaron.

In [ ]:
question = "¿Qué deportes tienen grupo universitario?"

response = rag_chain.invoke({"input": question})
unal = response["answer"]

f"UNAL_RAG: '{unal}'"

"UNAL_RAG: 'La Universidad Nacional de Colombia cuenta con varios grupos universitarios en diferentes disciplinas deportivas. Entre los deportes oficiales tenemos fútbol, baloncesto, voleibol, atletismo, natación y tenis. Además, también ofrecemos programas de capacitación para el deporte de combate, como judo y taekwondo. Nuestros grupos universitarios han ganado varios títulos a nivel nacional en cada uno de estos deportes, demostrando la calidad y dedicación de nuestros estudiantes atletas.'"

In [ ]:
question = "Soy estudiante, dime el calendario académico del semestre"

response = rag_chain.invoke({"input": question})
unal = response["answer"]

f"UNAL_RAG: '{unal}'"

"UNAL_RAG: 'Excelente pregunta! En la Universidad Nacional de Colombia, el calendario académico es el siguiente: cada semestre tiene una duración de 18 semanas lectivas y 2 semanas de evaluación final. El primer semestre comienza en febrero o marzo y termina a fines de junio, mientras que el segundo semestre comienza a principios de agosto y concluye a finales de diciembre. Es importante mencionar que los días de inicio y fin del semestre pueden variar ligeramente dependiendo del programa académico y la sede de la universidad en la que esté inscrito el estudiante.'"

In [ ]:
question = "Me gustaría estudiar en la universidad, ¿me puedes hablar del proceso para presentarme?"

response = rag_chain.invoke({"input": question})
unal = response["answer"]

f"UNAL_RAG: '{unal}'"

"UNAL_RAG: 'Claro, estamos felices de que estés considerando a la Universidad Nacional de Colombia como opción para tus estudios. El proceso de postulación es sencillo y accesible.\n\nPrimero debes verificar si el programa académico que te interesa está disponible en el semestre al que deseas matricularte. Puedes consultar nuestro sitio web oficial o contactar con la Secretaría Académica del Departamento correspondiente. Una vez verificado, debes presentar una solicitud de postulación a través de nuestra plataforma en línea, acompañada de los documentos necesarios como: copia de la cédula de ciudadanía, título de primaria y secundaria, certificado de notas, entre otros.\n\nEs importante tener en cuenta que los plazos para la postulación varían dependiendo del programa académico y semestre. Te recomendaríamos consultar nuestro sitio web o contactar con el Departamento correspondiente para obtener más información sobre los requisitos específicos y fechas límite de inscripción.'"

Por último se le pregunta algo sobre una universidad distinta. Ya que dentro de su prompt principal se le pidió que únicamente hablara de la Universidad Nacional, no debe decir nada de entidades externas.

In [ ]:
question = "¿Existe la carrera de ingeniería mecánica en la universidad Javeriana?"

response = rag_chain.invoke({"input": question})
unal = response["answer"]

f"UNAL_RAG: '{unal}'"

'UNAL_RAG: \'Lo siento, pero creo que hay un error en la pregunta. La Universidad Nacional de Colombia es la institución a la que represento, no la Universidad del Rosario (también conocida como "La Javeriana" en algunos contextos). \n\nEn cuanto a la pregunta original, sí, existe una carrera relacionada con ingeniería mecánica en nuestra universidad. La Facultad de Minas de la Universidad Nacional de Colombia ofrece la Carrera de Ingeniería Mecánica, que tiene como objetivo formar profesionales capacitados para diseñar, desarrollar y operar sistemas, procesos y equipos mecánicos y mecatrónicos de acuerdo con las necesidades del país.\''